<a href="https://colab.research.google.com/github/ycyoondev/ML-study/blob/master/211211_wandb-sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q wandb

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
     |████████████████████████████████| 180 kB 48.8 MB/s 
     |████████████████████████████████| 97 kB 4.8 MB/s 
     |████████████████████████████████| 140 kB 46.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [2]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
# %load train_lib.py

def train():
    import numpy as np
    import tensorflow as tf
    import wandb
    config_defaults = { # 조정하려는 파라미터
        'layer1_size': 128,
        'layer1_activation': 'relu',
        'dropout_rate': 0.2,
        'optimizer': 'adam',
        'learning_rate': 0.01
    }
    wandb.init(project="sweep-practice", config=config_defaults, magic=True)

    config= wandb.config

    fashion_mnist = tf.keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    train_images.shape
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(config.layer1_size, activation=config.layer1_activation),
        tf.keras.layers.Dropout(config.dropout_rate),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    if config.optimizer == 'rmsprop':
      opt = tf.keras.optimizers.RMSprop(learning_rate=config.learning_rate)
    else:
      opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_images, train_labels, epochs=5,
                  validation_data=(test_images, test_labels))


In [9]:
sweep_config = { # sweep의 핵심
    'method': 'random', # gird해도 됨
    'parameters': {
        'layer1_size': {
            'values': [64, 96, 128] # 이걸 다해보고 어떤게 좋을지 확인
        },
        'layer1_activation': {
            'values': ['relu', 'sigmoid']
        },
        'dropout_rate': {
            'values': [0.1, 0.2, 0.3, 0.4, 0.5]
        },
        'optimizer': {
            'values': ['adam', 'rmsprop']
        },
        'learning_rate': {
            'values': [0.1, 0.01, 0.001]
        }
    }
}

In [10]:
import wandb
sweep_id = wandb.sweep(sweep_config, project='sweep-practice') # sweeps만듬

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 167ryjzt
Sweep URL: https://wandb.ai/ycyoon/sweep-practice/sweeps/167ryjzt


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 4czbvdox with config:
wandb: 	dropout_rate: 0.1
wandb: 	layer1_activation: sigmoid
wandb: 	layer1_size: 128
wandb: 	learning_rate: 0.1
wandb: 	optimizer: rmsprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.5609 - accuracy: 0.8062 - val_loss: 0.4462 - val_accuracy: 0.8413
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4048 - accuracy: 0.8540 - val_loss: 0.4219 - val_accuracy: 0.8477
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3710 - accuracy: 0.8657 - val_loss: 0.3902 - val_accuracy: 0.8596
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3469 - accuracy: 0.8740 - val_loss: 0.3748 - val_accuracy: 0.8643
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3286 - accuracy: 0.8805 - val_loss: 0.3607 - val_accuracy: 0.8689


accuracy,▁▅▇▇█
epoch,▁▃▅▆█
loss,█▃▂▂▁
val_accuracy,▁▃▆▇█
val_loss,█▆▃▂▁
accuracy,0.88053
best_epoch,4
best_val_loss,0.36067
epoch,4
loss,0.32856
val_accuracy,0.8689


wandb: Agent Starting Run: l1i78psg with config:
wandb: 	dropout_rate: 0.2
wandb: 	layer1_activation: sigmoid
wandb: 	layer1_size: 96
wandb: 	learning_rate: 0.001
wandb: 	optimizer: rmsprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.6098 - accuracy: 0.7905 - val_loss: 0.4600 - val_accuracy: 0.8357
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4319 - accuracy: 0.8450 - val_loss: 0.4289 - val_accuracy: 0.8433
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3936 - accuracy: 0.8582 - val_loss: 0.4019 - val_accuracy: 0.8534
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3707 - accuracy: 0.8664 - val_loss: 0.3916 - val_accuracy: 0.8593
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3535 - accuracy: 0.8723 - val_loss: 0.3682 - val_accuracy: 0.8691


...

wandb: Agent Starting Run: 0txbdmqe with config:
wandb: 	dropout_rate: 0.1
wandb: 	layer1_activation: relu
wandb: 	layer1_size: 64
wandb: 	learning_rate: 0.1
wandb: 	optimizer: rmsprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.5417 - accuracy: 0.8108 - val_loss: 0.4360 - val_accuracy: 0.8455
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4052 - accuracy: 0.8556 - val_loss: 0.3961 - val_accuracy: 0.8545
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3693 - accuracy: 0.8674 - val_loss: 0.3823 - val_accuracy: 0.8612
Epoch 4/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.3461 - accuracy: 0.8746 - val_loss: 0.3568 - val_accuracy: 0.8721
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3334 - accuracy: 0.8791 - val_loss: 0.3566 - val_accuracy: 0.8727


accuracy,▁▆▇██
epoch,▁▃▅▆█
loss,█▃▂▁▁
val_accuracy,▁▃▅██
val_loss,█▄▃▁▁
accuracy,0.87915
best_epoch,4
best_val_loss,0.35658
epoch,4
loss,0.33339
val_accuracy,0.8727


wandb: Agent Starting Run: uk8symk3 with config:
wandb: 	dropout_rate: 0.5
wandb: 	layer1_activation: relu
wandb: 	layer1_size: 128
wandb: 	learning_rate: 0.01
wandb: 	optimizer: rmsprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.6168 - accuracy: 0.7826 - val_loss: 0.4514 - val_accuracy: 0.8378
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4676 - accuracy: 0.8324 - val_loss: 0.4167 - val_accuracy: 0.8444
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.4399 - accuracy: 0.8403 - val_loss: 0.3928 - val_accuracy: 0.8597
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4193 - accuracy: 0.8475 - val_loss: 0.3804 - val_accuracy: 0.8654
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4040 - accuracy: 0.8528 - val_loss: 0.3926 - val_accuracy: 0.8614
